In [2]:
import yfinance as yf

In [3]:
aapl = yf.Ticker("AAPL")

In [4]:
data = yf.download("AAPL", interval="1d", start="1999-12-31", end="2014-12-31")

[*********************100%***********************]  1 of 1 completed


In [6]:
data["Close"]

Date
1999-12-30     0.895647
1999-12-31     0.917969
2000-01-03     0.999442
2000-01-04     0.915179
2000-01-05     0.928571
                ...    
2014-12-23    28.135000
2014-12-24    28.002501
2014-12-26    28.497499
2014-12-29    28.477501
2014-12-30    28.129999
Name: Close, Length: 3774, dtype: float64

In [7]:
data = yf.download("YNDX", interval="1d", start="1999-12-31", end="2016-12-31")

[*********************100%***********************]  1 of 1 completed


In [10]:
series = data["Close"]

In [17]:
import pandas as pd
import numpy as np
def transform_to_DOIU(prices):
    arr = pd.DataFrame(prices)[0].pct_change().to_numpy()[1:]
    x = [1]
    for el in arr:
        x.append(x[-1]*(1+el))
    return np.array(x)


In [18]:
tr_series = transform_to_DOIU(series.values)

In [27]:
OUTPUT_WINDOW_SIZE = 30
INPUT_WINDOW_SIZE = 60
prices = series.values

X = []
y = []
for i in range(0, len(prices), OUTPUT_WINDOW_SIZE):

    tr_x = transform_to_DOIU(prices[i:INPUT_WINDOW_SIZE+i
                                    + OUTPUT_WINDOW_SIZE])
    y.append(tr_x[-1])
    X.append(tr_x[:INPUT_WINDOW_SIZE])

X = X[:-1]
y = y[:-1]

In [30]:
len(X), len(y)

(47, 47)

In [36]:
def prepare_data(prices, OUTPUT_WINDOW_SIZE = 30, INPUT_WINDOW_SIZE = 60):
    X = []
    y = []
    for i in range(0, len(prices), OUTPUT_WINDOW_SIZE):

        tr_x = transform_to_DOIU(prices[i:INPUT_WINDOW_SIZE+i
                                        + OUTPUT_WINDOW_SIZE])
        y.append(tr_x[-1])
        X.append(tr_x[:INPUT_WINDOW_SIZE])

    X = X[:-1]
    y = y[:-1]
    return X, y

In [37]:
X, y = prepare_data(series.values)

In [39]:
len(X)

47